## Loading the Dataset

In [ ]:
!wget --header="Host: doc-00-9g-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: AUTH_42pb7269gjo3pha4td7asmuk2haom6dp_nonce=cimfihe9v2vsq" --header="Connection: keep-alive" "https://doc-00-9g-docs.googleusercontent.com/docs/securesc/4qne5atjaj0oevah0md15eo25jnp41c3/7mjlfjgsba02rrfsopbf9uen8d1ap5k3/1617703350000/01699190096502596233/01699190096502596233/1mePcf4jG0RSCdwsPk_ueNqnmecpXUZuK?e=download&authuser=1&nonce=cimfihe9v2vsq&user=01699190096502596233&hash=nou9pos54g3g1l9sc24oa68i1mgj3qqq" -c -O 'dataset.csv'

--2021-04-06 10:03:16--  https://doc-00-9g-docs.googleusercontent.com/docs/securesc/4qne5atjaj0oevah0md15eo25jnp41c3/7mjlfjgsba02rrfsopbf9uen8d1ap5k3/1617703350000/01699190096502596233/01699190096502596233/1mePcf4jG0RSCdwsPk_ueNqnmecpXUZuK?e=download&authuser=1&nonce=cimfihe9v2vsq&user=01699190096502596233&hash=nou9pos54g3g1l9sc24oa68i1mgj3qqq
Resolving doc-00-9g-docs.googleusercontent.com (doc-00-9g-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to doc-00-9g-docs.googleusercontent.com (doc-00-9g-docs.googleusercontent.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘dataset.csv’

dataset.csv             [     <=>            ]  81.60M  62.7MB/s    in 1.3s    

2021-04-06 10:03:18 (62.7 MB/s) - ‘dataset.csv’ saved [85560694]



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
import xgboost as xg 
from mlxtend.regressor import StackingCVRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df1=pd.read_csv("dataset.csv")

In [ ]:
df1.head()

,segment_id,sensor_1_sum,sensor_1_mean,sensor_1_max,sensor_1_min,sensor_1_std,sensor_1_diff_min_max,sensor_1_diff_mean_max,sensor_1_quantile25,sensor_1_quantile50,sensor_1_quantile75,sensor_1_quantile99,sensor_1_mean_10000,sensor_1_mean_10000l,sensor_1_std_10000,sensor_1_std_10000l,sensor_2_sum,sensor_2_mean,sensor_2_max,sensor_2_min,sensor_2_std,sensor_2_diff_min_max,sensor_2_diff_mean_max,sensor_2_quantile25,sensor_2_quantile50,sensor_2_quantile75,sensor_2_quantile99,sensor_2_mean_10000,sensor_2_mean_10000l,sensor_2_std_10000,sensor_2_std_10000l,sensor_3_sum,sensor_3_mean,sensor_3_max,sensor_3_min,sensor_3_std,sensor_3_diff_min_max,sensor_3_diff_mean_max,sensor_3_quantile25,sensor_3_quantile50,...,sensor_1_rolling_std_100,sensor_1_rolling_max_100,sensor_1_rolling_min_100,sensor_2_rolling_mean_100,sensor_2_rolling_std_100,sensor_2_rolling_max_100,sensor_2_rolling_min_100,sensor_3_rolling_mean_100,sensor_3_rolling_std_100,sensor_3_rolling_max_100,sensor_3_rolling_min_100,sensor_4_rolling_mean_100,sensor_4_rolling_std_100,sensor_4_rolling_max_100,sensor_4_rolling_min_100,sensor_5_rolling_mean_100,sensor_5_rolling_std_100,sensor_5_rolling_max_100,sensor_5_rolling_min_100,sensor_6_rolling_mean_100,sensor_6_rolling_std_100,sensor_6_rolling_max_100,sensor_6_rolling_min_100,sensor_7_rolling_mean_100,sensor_7_rolling_std_100,sensor_7_rolling_max_100,sensor_7_rolling_min_100,sensor_8_rolling_mean_100,sensor_8_rolling_std_100,sensor_8_rolling_max_100,sensor_8_rolling_min_100,sensor_9_rolling_mean_100,sensor_9_rolling_std_100,sensor_9_rolling_max_100,sensor_9_rolling_min_100,sensor_10_rolling_mean_100,sensor_10_rolling_std_100,sensor_10_rolling_max_100,sensor_10_rolling_min_100,time_to_eruption
0,698305523,200457.0,3.340894,2931.0,-3019.0,367.633890,-88.0,2927.659106,-214.0,0.0,219.0,970.0,4.9501,1.0707,432.279698,284.327545,139207.0,2.320078,4738.0,-6015.0,514.740948,-1277.0,4735.679922,-245.0,0.0,250.0,1456.0,9.9759,-31.2292,607.668508,376.987984,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,12560.427735,101000.416381,-100518.814577,109.801028,7318.858618,82460.650587,-65971.628929,0.000000,0.000000,0.000000,0.000000,-101.702544,9658.821046,37860.546906,-41859.374593,-18.111054,2198.300876,25286.849071,-30101.034547,770.053448,52707.620672,165219.792330,-145440.692262,-413.937449,18213.129008,83060.769122,-68436.016554,90.522089,26553.606751,90926.196470,-90599.871335,0.000000,0.000000,0.000000,0.000000,65.655065,15043.202930,114344.945877,-115229.604770,13903256
1,1191644191,-165217.0,-2.753571,4223.0,-4154.0,611.311853,69.0,4220.246429,-376.0,0.0,369.0,1557.0,-3.4140,-1.6858,766.909079,530.485286,-8315.0,-0.138581,20325.0,-11098.0,1140.506282,9227.0,20324.861419,-582.0,0.0,582.0,3057.0,-1.1737,-3.3488,1485.371941,1047.332342,-3407912.0,-56.797587,4863.0,-4617.0,1213.212902,246.0,4806.202413,-776.0,0.0,...,17872.470064,63185.450933,-75414.603451,-7.568476,13309.511555,162725.319550,-213930.268345,-2787.964789,56027.833712,190487.754195,-204473.438197,-66.206437,16822.822031,61979.902191,-61442.031437,-2.798846,473.047632,9352.818705,-7071.722522,751.287779,41078.410030,135828.790562,-126549.163249,372.027733,31667.498627,128111.273316,-97630.384883,206.703481,37717.767319,111428.176279,-150875.433831,52.105823,12029.097913,70767.791070,-71823.590341,499.870725,30508.686691,126078.853724,-164003.213363,5525566
2,1403440092,-95822.0,-1.597007,2118.0,-1918.0,306.984282,200.0,2116.402993,-184.0,0.0,186.0,748.0,-2.0455,-4.0018,288.242237,269.900865,-10101.0,-0.168347,8604.0,-6475.0,514.620059,2129.0,8603.831653,-254.0,0.0,249.0,1369.0,-1.2282,-0.0584,454.868550,333.709113,-192776.0,-3.212880,1335.0,-1646.0,259.603885,-311.0,1331.787120,-165.0,0.0,...,8400.388923,70925.556339,-57435.023158,-13.389881,5652.716551,57149.347197,-38962.240673,-150.096677,7317.215505,36723.636479,-39184.035652,19.749261,6584.465991,37627.717487,-33677.438245,-44.171727,2082.616831,26549.361167,-28961.676900,96.706874,23969.962904,72166.436490,-80872.28

In [ ]:
Y=df1["time_to_eruption"]
X=df1.drop(columns=['time_to_eruption',"segment_id"])

## With All Features

### Splitting the dataset

In [ ]:
Y

0       13903256
1        5525566
2        4524342
3       19863157
4       14951435
          ...   
4426    36335121
4427     3819695
4428    32331124
4429    24993068
4430     5288221
Name: time_to_eruption, Length: 4431, dtype: int64

In [ ]:
#80-20 split
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Normalising the Dataset

In [ ]:
# Since the range of values across different columns are different we are Normalizing the Dataset 
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# https://www.geeksforgeeks.org/standardscaler-minmaxscaler-and-robustscaler-techniques-ml/
scaler=MinMaxScaler()
scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(scaled, columns =X_train.columns.values )
scaled = scaler.transform(X_test)
X_test = pd.DataFrame(scaled, columns =X_test.columns.values )

In [ ]:
#50-50 split
D1,D2,Y1,Y2 = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [ ]:
print(D1.shape)
print(D2.shape)
print(X_test.shape)

(2480, 1100)
(1064, 1100)
(887, 1100)


### Tuning The Base And Meta Models

In [ ]:
# initialising the models
dcsn = DecisionTreeRegressor()
dcsn1 = DecisionTreeRegressor()

rf = RandomForestRegressor()

In [ ]:
#Intialising Stack CV Regressor
#https://rasbt.github.io/mlxtend/user_guide/regressor/StackingCVRegressor/
stack = StackingCVRegressor(regressors=(dcsn,dcsn1),meta_regressor=rf,use_features_in_secondary=True)

### Stack Regressor

In [ ]:
#Getting all the parameters
stack.get_params().keys()

dict_keys(['decisiontreeregressor-1', 'decisiontreeregressor-2', 'decisiontreeregressor-1__ccp_alpha', 'decisiontreeregressor-1__criterion', 'decisiontreeregressor-1__max_depth', 'decisiontreeregressor-1__max_features', 'decisiontreeregressor-1__max_leaf_nodes', 'decisiontreeregressor-1__min_impurity_decrease', 'decisiontreeregressor-1__min_impurity_split', 'decisiontreeregressor-1__min_samples_leaf', 'decisiontreeregressor-1__min_samples_split', 'decisiontreeregressor-1__min_weight_fraction_leaf', 'decisiontreeregressor-1__presort', 'decisiontreeregressor-1__random_state', 'decisiontreeregressor-1__splitter', 'decisiontreeregressor-2__ccp_alpha', 'decisiontreeregressor-2__criterion', 'decisiontreeregressor-2__max_depth', 'decisiontreeregressor-2__max_features', 'decisiontreeregressor-2__max_leaf_nodes', 'decisiontreeregressor-2__min_impurity_decrease', 'decisiontreeregressor-2__min_impurity_split', 'decisiontreeregressor-2__min_samples_leaf', 'decisiontreeregressor-2__min_samples_spli

In [ ]:
#Initialising the Gridsearch for tuning
grid = GridSearchCV(estimator=stack, param_grid={
          'decisiontreeregressor-1__min_weight_fraction_leaf': [0.01, 0.05, 0.1],
          'decisiontreeregressor-2__min_weight_fraction_leaf': [0.01, 0.05, 0.1],
          'meta-randomforestregressor__min_samples_split': [2,5,10]
    }, 
    cv=3,
    refit=True
)

In [ ]:
D11=D1.to_numpy()

In [ ]:
Y11=Y1.to_numpy()

In [ ]:
#Tuning the model 
grid.fit(D11, Y11)

GridSearchCV(cv=3, error_score=nan,
             estimator=StackingCVRegressor(cv=5,
                                           meta_regressor=RandomForestRegressor(bootstrap=True,
                                                                                ccp_alpha=0.0,
                                                                                criterion='mse',
                                                                                max_depth=None,
                                                                                max_features='auto',
                                                                                max_leaf_nodes=None,
                                                                                max_samples=None,
                                                                                min_impurity_decrease=0.0,
                                                                                min_impurity_split=None,
                   

In [ ]:
#Parameters after tuning
grid.best_params_

{'decisiontreeregressor-1__min_weight_fraction_leaf': 0.1,
 'decisiontreeregressor-2__min_weight_fraction_leaf': 0.1,
 'meta-randomforestregressor__min_samples_split': 5}

In [ ]:
#Initialising the best models
dcsn_bst = DecisionTreeRegressor(min_weight_fraction_leaf= 0.1)
dcsn_bst1 = DecisionTreeRegressor(min_weight_fraction_leaf= 0.1)

rf_bst = RandomForestRegressor(min_samples_split= 5)

In [ ]:
#Stack Model with best parameters
stack_bst = StackingCVRegressor(regressors=(dcsn_bst,dcsn_bst1),meta_regressor=rf,use_features_in_secondary=True)

In [ ]:
stack_bst.fit(D11,Y11)

StackingCVRegressor(cv=5,
                    meta_regressor=RandomForestRegressor(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         criterion='mse',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                           

In [ ]:
y_pred=stack_bst.predict(D11)

In [ ]:
#Train RMSE
score=mean_absolute_error(Y11 , y_pred )
score

2041611.8890120967

In [ ]:
y_pred=stack_bst.predict(D2)

In [ ]:
#Validation RMSE
score=mean_absolute_error(Y2 , y_pred )
score

4146931.909342105

In [ ]:
y_pred=stack_bst.predict(X_test)

In [ ]:
#Test RMSE
score=mean_absolute_error(y_test , y_pred )
score

3746281.017113867

## Observation

>Model | Train RMSE | CV RMSE | Test RMSE 
>---|---|:---|:---:
>Linear Regression|3658210.004938272|8492927.304654444|8490458.537767757
>Lasso Regression|3846744.970136143|6492706.85465377|6888134.921417941
>Ridge Regression|4335903.838483236|5631265.087146504|5586910.474889704
>Support Vector Regression|3467886.9368207967|5143544.124513431|5086226.870055837
>Decision Tree Regression |4717354.729805069|5766288.416692352|5542370.67957373
>LGBM Regression|3497329.3607941954|4530130.518221223|4464948.321870959
>Custom Ensemble||3653445.2630618704|5292222.674077656
>StackCvRegressor|2041611.8890120967|4146931.909342105|3746281.017113867
